In [1]:
####################################
## A. Librerias
####################################
import os
import time
import json

import datetime
import pandas as pd

from utils.utils import *
from utils.paths import CHROMEDRIVER_PATH, PROJECT_PATH

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def start_har(proxy, name):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
    }
    proxy.new_har(name, options=options)

ESTAMOS EN --->  darwin
PROJECT_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper
CHROME_DRIVER_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver



In [2]:
########################
## C. Ejecución
########################

# 1. Seleccionar modo
HEADLESS = False
PRINT_VIEW = False

# 2. Eliminar todos los archivos de la carpeta
try:
    files = get_files(os.path.join(os.getcwd(),'outputs','jsons','renacyt_profile'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

Se eliminaron los archivos de la carpeta


In [3]:
######################
## D. Leer los jsons
######################

# 1. Leer los jsons y guardarlos en un dataframe
paths = get_files(os.path.join(os.getcwd(),'outputs','jsons','renacyt'), '.json')
df_renacyt = pd.concat([pd.read_json(path) for path in paths], axis=0)

df_renacyt.to_parquet(os.path.join(os.getcwd(),'outputs','parquets','renacyt.parquet'))
df_renacyt.astype(str).to_excel(os.path.join(os.getcwd(),'outputs','excels','renacyt.xlsx'), index=False)

# 2. Generar lista de urls a partir de ['ctiVitae']
urls = []
cti_vitaes = df_renacyt['ctiVitae'].tolist()
for cit_vitae in cti_vitaes:
    url = f'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador={cit_vitae}'
    urls.append(url)

urls = list(set(urls))
print(f'Se generaron {len(urls)} urls')

urls = urls[0:10]
urls

Se generaron 7152 urls


['https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=117300',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=3626',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=200850',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=15363',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=3184',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=36135',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=97804',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=2067',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=15389',
 'https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=38519']

In [10]:
######################
## E. Ejecutar
######################

# 1. Iniciar el driver, el servidor proxy y el proxy
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)

# 2. Iniciar har
start_har(proxy, 'renacyt_profile')

# 3. Iterar sobre las urls
i = 0
for url in urls: 
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "elementor-image")))
        print(f'{i+1}. {url}')
        i += 1
    except:
        break

Se inicia el servidor en Mac o Linux
Usando el chromedriver_path: /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver
1. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=117300
2. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=3626
3. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=200850
4. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=15363
5. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=3184
6. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=36135
7. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=97804
8. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=2067
9. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=15389
10. https://servicio-renacyt.concytec.gob.pe/ficha-renacyt/?idInvestigador=38519


In [11]:
# 4. Obtener el .har y guardar los json
time.sleep(TIME_SLEEP*2)
har = proxy.har

In [16]:
i = 0
for entrie in har.get('log').get('entries'):
    aux_entrie = str(entrie)

    # data_list = json.loads(string_content)
    data_list = aux_entrie
    with open(os.path.join(os.getcwd(),'outputs','jsons','renacyt_profile','profile_' + str(i).zfill(2) + '_data.json'), 'w') as json_file:
        json.dump(data_list, json_file, indent=4)
    i += 1

In [6]:
stop_chrome_driver(driver, server, proxy)

# Eliminar los archivos bmp.log y server.log
try:
    os.remove('bmp.log')
    os.remove('server.log')
    print("Se eliminaron los archivos bmp.log y server.log")
except:
    pass

Se eliminaron los archivos bmp.log y server.log
